In [160]:
import urllib
import urllib2
import oauth2

from bs4 import BeautifulSoup
import lxml
import string
import pandas as pd
import numpy as np

import pymysql
#from KAsql import query_SQL

In [178]:
# class ReturnValue(object):
#     # Facilitates returning two values without a tuple or dict. Instead returns a class object
#     def __init__(self, html, filename):
#         self.html = html
#         self.filename = filename
#         print(self.filename)

In [179]:
def get_html(yelp_id):
    yelp_url = 'http://www.yelp.com/biz/'+yelp_id
    #print(yelp_url)
    response = urllib2.urlopen(yelp_url)
    html_obj = response.read()
    return html_obj

In [180]:
def save_myhtml(html_filename, html):
    # Save the html file
    with open(html_filename, 'wb') as f:
        f.write(html)

In [181]:
def format_html(string,int):
    # accepts a rating or date and returns a float btw 0.0 and 5.0, or date-string
    s = str(string)
    s = s.lstrip('[<meta content="')
    if int==0:
        s = float(s[0:3])#star ratings are always 3 characters long
    elif int==1:
        stop = s.find('"')#date lengths could vary
        s = s[0:stop]
    else: 
        print('Wrong integer given to format_rating. Must be 0 for rating or 1 for date: ',int)
    return s

In [182]:
# not working i don't know why
def remove_backslash(review):
    #review = review.replace(u'\xa0','')
    review = review.replace(u"\\",'')#This doesn't seem to work...
    review = review.replace(u"/",'')
    review = review.replace(u"(",'')
    review = review.replace(u")",'')
    review = review.replace(u"-",'')
    review = review.replace(u'"','')
    return review

In [210]:
def scrape_reviews(html):
    
    # Save a temporary file - BeautifulSoup was not running well when passed an html object, needs a saved page
    html_filename = 'temp.html'
    save_myhtml(html_filename, html)
    # open the html file/object with beautiful soup
    soup = BeautifulSoup(open(html_filename), "lxml")

    #pp.pprint(ptext)#<class 'bs4.element.ResultSet'>
    ptext = soup.find_all('p', {'itemprop': 'description'})
    reviews = [ptext[i].get_text() for i in range(0,len(ptext))]
    reviews = [filter(lambda x: x in string.printable, review) for review in reviews]
    reviews = [remove_backslash(review) for review in reviews]
    
    # Examples of what the html looks like:
    # <meta itemprop="ratingValue" content="5.0">
    # <meta itemprop="datePublished" content="2013-03-27">

    # R-Stars and Dates: Note that R-stars are integers while B-stars (average of R-stars are rounded to X.5)
    rstars =[];rdates=[]# Currently dates aren't going into the master database - no time for that
    rstars =[pt.parent.find_all('meta', {'itemprop':'ratingValue'}) for pt in ptext]
    rstars =[format_html(rstar,0) for rstar in rstars]
    #rdates = [pt.parent.find_all('meta', {'itemprop':'datePublished'}) for pt in ptext]
    #rdates =[format_html(rdate,1) for rdate in rdates]

    # There's an overall rating on this page I could also get, but it should match bstars, and would therefore be redundant (hopefully)

    # Ensure that ratings and reviews are vectors of the same length
    if len(rstars)!=len(reviews):
        print('Length of rstars and reviews on this page do not match.')
        #print('rstars len: ',len(rstars),' reviews len: ',len(reviews))
        rmin = min(len(rstars),len(reviews))
        rstars = rstars[0,rmin]; reviews = reviews[0,rmin]; rdates = rdates[0,rmin]

    #Here I manually verified that the star-ratings, reviews, and dates match up with a few pages
#     for i in range(0,len(reviews)):
#         print(rstars[i],rdates[i],reviews[i])
        
    return [rstars], [reviews]

In [205]:
def query_SQL():
    # Get fields from business table that need to be used to populate reviews table (and also are reproduced in reviews table for convenience)
    con = False
    rows = []
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='ortho')
        with con:
            cur = con.cursor()
            sql = 'USE ortho;'
            cur.execute(sql)
            #sql = 'SELECT id,term_id,city_id FROM business WHERE yelp_id = "'+yelp_business_id+'";' 
            sql = 'SELECT id, yelp_id, term_id, city_id FROM business;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
            #print('business id from table: ',rows, type(rows))
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [206]:
def insert_reviews_SQL(business_id, rstar, review):
    con = False
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            #print('inserting into review db')
            #print(type(business_id), type(rstar), type(review))
            myvalues = "'" + str(business_id)+"'," + str(rstar) + "," + '"'+review+'"'
            sql = 'INSERT INTO review(business_id,stars,comment) VALUES('+myvalues+')'
            #print(sql)
            cur.execute(sql)
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()

In [218]:
def main():
    
    #from ortho.business, returns: id, yelp_id, term_id, city_id (tuple in a tuple)
    rows = query_SQL()#response is shape rev# x 4
    #print rows[0][:]
    
    # UNPACK ROWS - no matter which order you place these indices, if one is a : it will give you everything in the row indexed by the number you gave 
    bid=[];yid=[];tid=[];cid=[];
    for row in rows:
        #print row[0],row[1],row[2]
        bid.append(row[0])
        yid.append(row[1])
        tid.append(row[2])
        cid.append(row[3])
    print(bid[:5],yid[:5])
    
    # Make Pandas DataFrame
    df = pd.DataFrame([bid,yid,tid,cid]).transpose()
    df.columns=['bid','yelp_id','term_id','city_id']
    print '\n',df.head(5)
    
    # Save a temporary html file: soup breaks if you pass it html obj
#     htmls =[]; html_filenames=[];
#     htmls = [get_html(id) for id in yid[:3]]
    #html_filenames = [(id +'.html') for id in yid[:3]]
    #print html_filenames
    
#     temp = 'james-l-chen-md-san-francisco'
#     html = get_html(temp)
    #save_myhtml(html_filename, html)
    
    #for i in range(0,df.shape[0]):# loop over each business
    #for i in range(0,3):# for debugging: loop over 1 business
        
    #Get reviews & their stars - Beautiful Soup scrape
    html_filename = 'temp.html'#I don't want to keep all the html files in my folder, but I need a replacable saved file in order for Beautiful Soup to work correctly
    for row in df.loc[0:3,:]
        print 'row = ',row
        html = get_html(row[1])#yelp_id as argument
        save_myhtml(html_filename, html)
        rstars,reviews = scrape_reviews(html)
        print rstars,reviews
        #print('SQL review input: ',business_id[0], rstars[0][i],reviews[0][i[:10]])
        #insert_reviews_SQL(business_id[0][0], rstars[0][i], reviews[0][i])
        
        
    #for i in range(0,np.shape(rstars)[1]):#np.shape of rstars is (1,40)
    #for i in range(0,1):#np.shape of rstars is (1,40)
        #print('SQL review input: ',business_id[0], rstars[0][i],reviews[0][i[:10]])
        #insert_reviews_SQL(business_id[0][0], rstars[0][i], reviews[0][i])
            
            #x = np.asarray([[business_id]*40]).T
            #df = pd.DataFrame(rstars,reviews)
            #print(df)

#         DEGUGGING CODE ONLY
#         temp_url = 'total-care-plus-san-francisco.html'
#         temp_url = 'dan-kalshan-md-san-francisco-2.html'
    
if __name__ == '__main__':
    main()

([3, 4, 5, 6, 7], ['james-l-chen-md-san-francisco', 'elly-s-laroque-md-san-francisco', 'nicholas-colyvas-md-san-francisco', 'c-benjamin-ma-md-san-francisco', 'irene-minkowsky-md-san-francisco'])

  bid                            yelp_id term_id city_id
0   3      james-l-chen-md-san-francisco       1       1
1   4    elly-s-laroque-md-san-francisco       1       1
2   5  nicholas-colyvas-md-san-francisco       1       1
3   6     c-benjamin-ma-md-san-francisco       1       1
4   7   irene-minkowsky-md-san-francisco       1       1
row =  (0, bid                                    3
yelp_id    james-l-chen-md-san-francisco
term_id                                1
city_id                                1
Name: 0, dtype: object)


TypeError: cannot concatenate 'str' and 'int' objects